## **Sentiment Analysis on Moroccan Arabic Dialect**


> Mohamed Amine EL YAGOUBY





In [ ]:
pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 6.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import gensim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, Dropout, LSTM, Masking,Bidirectional
from keras.optimizers import Adam
from keras.regularizers import l2
from gensim.models import Word2Vec
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
import re
import emoji
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
import nltk
import string
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Data Preprocessing

In [ ]:
def preprocess_arabic_text(text, stop_words):
    text = re.sub(r'\d+', '', text)
    text = ''.join(c for c in text if c not in emoji.EMOJI_DATA)
    text = text.replace('\n', ' ')
    text = text.lower()
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    text = ' '.join(filtered_tokens)
    return text

with open('/content/drive/MyDrive/sentiment-analysis-on-moroccan-arabic-dialect/list.txt', 'r', encoding='utf-8') as file:
    stop_words = file.read().splitlines()


data = pd.read_csv('/content/drive/MyDrive/sentiment-analysis-on-moroccan-arabic-dialect/train.csv', index_col=0)

data['comment'] = data['comment'].apply(lambda x: preprocess_arabic_text(x, stop_words))

In [ ]:
stop_words = set(stopwords.words('arabic'))

def remove_diacritics(text):
    arabic_diacritics = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(arabic_diacritics, '', str(text))
    return text

def remove_emoji(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def clean_text(text):
    text = "".join([word for word in text if word not in string.punctuation])
    text = remove_emoji(text)
    text = remove_diacritics(text)
    text = re.sub(r'[.,\'"()؛!؟“”‘’….]', '', text)
    tokens = word_tokenize(text)
    text = ' '.join([word for word in tokens if word not in stop_words])
    return text
data['comment'] = data['comment'].apply(clean_text)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data['comment'], data['label'], test_size=0.2, random_state=42)

# Word Embeddings with Word2Vec

In [ ]:
# Tokenize the data
tokenized_comments = [word_tokenize(comment) for comment in X_train]
tokenized_test_comments = [word_tokenize(comment) for comment in X_test]

In [ ]:
word2vec_model = gensim.models.Word2Vec(sentences=tokenized_comments + tokenized_test_comments,
                                        vector_size=200,  # Set the size of the word vectors
                                        window=5,
                                        min_count=3,
                                        workers=4)

# Training the Word2Vec model
word2vec_model.train(X_train, total_examples=len(X_train), epochs=100)

(1700, 25483500)

In [ ]:
print("Some words from the vocabulary:")
print(list(word2vec_model.wv.index_to_key[:100]))

Some words from the vocabulary:
['التلقيح', 'اللقاح', 'الله', 'الناس', 'لقاح', 'جواز', 'تلقيح', 'هاد', 'المغرب', 'انا', 'راه', 'الجرعة', 'كورونا', 'الدولة', 'يجب', 'الفيروس', 'الملقحين', 'باش', 'اللقاحات', 'المغاربة', 'اللي', 'الصحة', 'المناعة', 'لله', 'الشعب', 'المواطنين', 'للتلقيح', 'ديال', 'شي', 'فرض', 'الجواز', 'العالم', 'الثالثة', 'الوباء', 'اللجنة', 'شيء', 'غادي', 'الحمد', 'الحكومة', 'الدول', 'واش', 'والله', 'علينا', 'المواطن', 'ملقح', 'حنا', 'بدون', 'يحمي', 'ماشي', 'هدا', 'الملقح', 'علاش', 'العلمية', 'العدوى', 'الملك', 'مناعة', 'لينا', 'يعني', 'جونسون', 'الجماعية', 'جرعة', 'فعال', 'حالة', 'الصيني', 'الجميع', 'سينوفارم', 'العمومية', 'عليكم', 'عملية', 'الغير', 'المغربي', 'الطبيعية', 'الصحي', 'رغم', 'عليهم', 'الأولى', 'المرض', 'لان', 'صحة', 'ملقحين', 'كلشي', 'كون', 'كوفيد', 'بالتلقيح', 'الصحية', 'البلاد', 'الحق', 'حاجة', 'بلا', 'ناس', 'المسؤولية', 'شخص', 'قرار', 'بالنسبة', 'فايزر', 'خطيرة', 'جانبية', 'اجباري', 'فعالية', 'للقاح']


In [ ]:
X_train_seq = [
    [word2vec_model.wv.key_to_index[word] for word in tokenized_comment if word in word2vec_model.wv.key_to_index]
    for tokenized_comment in tokenized_comments
]


# Pad sequences to have a consistent length
X_train_seq = pad_sequences(X_train_seq, maxlen=max(len(comment) for comment in tokenized_comments), padding='post')

In [ ]:
# Assuming tokenized_test_comments is a list of lists where each inner list contains tokenized words
X_test_seq = [
    [word2vec_model.wv.key_to_index[word] for word in tokenized_comment if word in word2vec_model.wv.key_to_index]
    for tokenized_comment in tokenized_test_comments
]

X_test_seq = pad_sequences(X_test_seq, maxlen=max(len(comment) for comment in tokenized_comments), padding='post')

# Model Building and Training

In [ ]:
model_lstm = Sequential()

model_lstm.add(Embedding(input_dim=len(word2vec_model.wv.index_to_key) + 1,
                         output_dim=100,
                         input_length=X_train_seq.shape[1],
                         mask_zero=True))

model_lstm.add(Bidirectional(LSTM(32, return_sequences=True, kernel_regularizer=l2(0.01))))
model_lstm.add(Flatten())
model_lstm.add(Dense(24, activation='relu'))
model_lstm.add(Dropout(0.7))
model_lstm.add(Dense(1, activation='sigmoid'))

In [ ]:
model_lstm.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [ ]:
model_lstm.fit(X_train_seq, y_train, epochs=5, batch_size=64,validation_data = (X_test_seq,y_test) ,verbose=1)

Epoch 1/5
24/24 [==============================] - 9s 378ms/step - loss: 0.3050 - accuracy: 0.9603 - val_loss: 1.1733 - val_accuracy: 0.7708
Epoch 2/5
24/24 [==============================] - 8s 321ms/step - loss: 0.2483 - accuracy: 0.9707 - val_loss: 1.0774 - val_accuracy: 0.7604
Epoch 3/5
24/24 [==============================] - 10s 412ms/step - loss: 0.1701 - accuracy: 0.9805 - val_loss: 1.1955 - val_accuracy: 0.7812
Epoch 4/5
24/24 [==============================] - 7s 307ms/step - loss: 0.1367 - accuracy: 0.9818 - val_loss: 1.4093 - val_accuracy: 0.7917
Epoch 5/5
24/24 [==============================] - 9s 384ms/step - loss: 0.1056 - accuracy: 0.9831 - val_loss: 1.5389 - val_accuracy: 0.7865


# Sentiment Prediction

In [ ]:
sentences = [
    'لقد كان يومًا رائعًا',
    'أنا حزين جدًا',
    'لقد فزت بالمباراة',
    'لقد خسرت وظيفتي',
    'لقد تخرجت من الجامعة',
    'لقد تزوجت',
    'لقد أنجبت طفلًا',
    'لقد مات والدي',
    'لقد انتقلت إلى منزل جديد'
]

preprocessed_sentences = [preprocess_arabic_text(sentence, stop_words) for sentence in sentences]
tokenized_sentences = [word_tokenize(sentence) for sentence in preprocessed_sentences]

sequences = [
    [word2vec_model.wv.key_to_index[word] for word in tokenized_sentence if word in word2vec_model.wv.key_to_index]
    for tokenized_sentence in tokenized_sentences
]

padded_sequences = pad_sequences(sequences, maxlen=max(len(comment) for comment in tokenized_comments), padding='post')

predictions = model_lstm.predict(padded_sequences)
predictions = np.round(predictions)

for sentence, prediction in zip(sentences, predictions):
    print(sentence, ':', prediction)

1/1 [==============================] - 6s 6s/step
لقد كان يومًا رائعًا : [1.]
أنا حزين جدًا : [1.]
لقد فزت بالمباراة : [1.]
لقد خسرت وظيفتي : [1.]
لقد تخرجت من الجامعة : [1.]
لقد تزوجت : [1.]
لقد أنجبت طفلًا : [1.]
لقد مات والدي : [0.]
لقد انتقلت إلى منزل جديد : [1.]
